# Red Neuronal Recurrente y Procesamiento del lenguaje Natural

In [5]:
# Librerias necesarias
import os, glob

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.data import Dataset
from tensorflow.keras import losses
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dense

In [6]:
data = pd.read_csv('datasets/nombres.csv')

### 1. Crear una lista con las palabras a generar

In [7]:
text = ' '.join([data.iloc[i, 0].lower() for i in data.index])
text

'alejandro valeria mateo sofía daniel camila andrés isabella sebastián mariana diego natalia gabriel fernanda samuel lucía leonardo victoria emiliano regina rodrigo ximena julián paulina nicolás renata ángel carolina tomás alejandra benjamín jimena jorge elisa adrián mónica luis gabriela ricardo andrea bruno claudia mauricio laura alan patricia iván teresa óscar beatriz aarón cecilia esteban silvia raúl rosa enrique julia marcos ángela rafael lourdes hugo fabiola sergio daniela arturo bianca eduardo pamela manuel araceli francisco verónica alonso miriam cristian karla joel yesenia martín rocío césar estrella israel danna kevin aitana jonathan zoe dante abril gael alma elías salma simón luna thiago aurora'

### 2. Preparar los datos y crear el vocabulario (preprocesamiento)

In [8]:
# Vocabulario
vocab = sorted(set(text))
char2idx = {c:i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

print(f'Vocabulario: {len(vocab)} caracteres únicos')
print(f'Caracteres: {vocab}')

Vocabulario: 31 caracteres únicos
Caracteres: [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'á', 'é', 'í', 'ó', 'ú']


In [9]:
# Ejemplo de nombre codificado
name = data.iloc[0, 0].lower()
print(f'Nombre: {name}')
print(f'Codificación: {[char2idx[c] for c in name]}')

Nombre: alejandro
Codificación: [1, 12, 5, 10, 1, 14, 4, 18, 15]


In [10]:
# Texto codificado
encoded = np.array([char2idx[c] for c in text])
print(encoded)

[ 1 12  5 10  1 14  4 18 15  0 22  1 12  5 18  9  1  0 13  1 20  5 15  0
 19 15  6 28  1  0  4  1 14  9  5 12  0  3  1 13  9 12  1  0  1 14  4 18
 27 19  0  9 19  1  2  5 12 12  1  0 19  5  2  1 19 20  9 26 14  0 13  1
 18  9  1 14  1  0  4  9  5  7 15  0 14  1 20  1 12  9  1  0  7  1  2 18
  9  5 12  0  6  5 18 14  1 14  4  1  0 19  1 13 21  5 12  0 12 21  3 28
  1  0 12  5 15 14  1 18  4 15  0 22  9  3 20 15 18  9  1  0  5 13  9 12
  9  1 14 15  0 18  5  7  9 14  1  0 18 15  4 18  9  7 15  0 23  9 13  5
 14  1  0 10 21 12  9 26 14  0 16  1 21 12  9 14  1  0 14  9  3 15 12 26
 19  0 18  5 14  1 20  1  0 26 14  7  5 12  0  3  1 18 15 12  9 14  1  0
 20 15 13 26 19  0  1 12  5 10  1 14  4 18  1  0  2  5 14 10  1 13 28 14
  0 10  9 13  5 14  1  0 10 15 18  7  5  0  5 12  9 19  1  0  1  4 18  9
 26 14  0 13 29 14  9  3  1  0 12 21  9 19  0  7  1  2 18  9  5 12  1  0
 18  9  3  1 18  4 15  0  1 14  4 18  5  1  0  2 18 21 14 15  0  3 12  1
 21  4  9  1  0 13  1 21 18  9  3  9 15  0 12  1 21

### 3. Crear datos para entrenar la red (secuencias)

In [11]:
seq_lenght = 16
char_dataset = Dataset.from_tensor_slices(encoded)
sequences = char_dataset.batch(seq_lenght+1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## 4. Crear red neuronal recurrente

In [ ]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return Sequential([
        Input(batch_shape=(batch_size, None)),
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])

model = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    BATCH_SIZE)

def loss(labels, logits):
    return losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
